In [2]:
import numpy as np # linear algebra
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import folium
import re

# This ensures Plotly output works in multiple places:
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "notebook_connected"


In [3]:
# Plotly: default template render
bnw = go.layout.Template(
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            showline=True,
            linecolor="black",
            linewidth=2,
            mirror=True,
            title=""
        ),
        margin=go.layout.Margin(
            l=70,
            r=30
        ),
        yaxis=go.layout.YAxis(
            showline=True,
            linecolor="black",
            linewidth=2,
            mirror=True,
            title=""
        ),
        font=go.layout.Font(
            # family="Old Standard TT",
            size=12
        ),
        title=go.layout.Title(
            font=go.layout.title.Font(
                # family="Old Standard TT",
                size=18
            )
        ),
        # paper_bgcolor="#eaeaf2"
#         font=dict(color="white"),
#         plot_bgcolor="black"
#         paper_bgcolor="black",
#         font=dict(color="white")
    ),
)

pio.templates["bnw"] = bnw

px.defaults.template = "bnw"

# All About Us -- ECE Fourth Year

Respondents: 52

In [3]:
df = pd.read_csv("Electrical_and_Computer_Engineering_2023_-_Class_Profile_Survey_Submissions_2023-03-29.csv")

In [4]:
# print(len(df))
# df.head(5)

## Data Cleaning

In [5]:
## Write out columns to a text file for analysis
with open("all_columns.txt", mode="w", encoding="utf-8") as f:
    f.write("\n".join(df.columns))

In [6]:
# Modify column data
df.rename(columns={
                    "Untitled long answer field (1)" : "1. Share a story, happy or sad, from your time in ECE",
                    "Untitled long answer field (2)" : "2. What is something you regret over your time at UofT?",
                    "Untitled long answer field (3)" : "3. Give a piece of advice to your fellow ECE 2T2 - 2023 grads"
                  }, inplace=True)

# Check column data has indeed been changed
index = df.columns.str.contains("1. Share a story, happy or sad, from your time in ECE") | \
df.columns.str.contains("2. What is something you regret over your time at UofT?") | \
df.columns.str.contains("3. Give a piece of advice to your fellow ECE 2T2 - 2023 grads")

indices = df.columns[index]
assert len(indices) == 3

In [7]:
# Drop unnecessary article. Non-useful data.

# Proof that all data under this column are null
assert df["https://money.usnews.com/money/personal-finance/family-finance/articles/where-do-i-fall-in-the-ameri"
          "can-economic-class-system"].isna().sum() == len(df)

df.drop("https://money.usnews.com/money/personal-finance/family-finance/articles/where-do-i-fall-in-the-american-economic-class-system", 
        axis=1, inplace=True)

## Data Exploration

## Top 5 Questions You Did Not Want to Answer

In [8]:
num_nulls = df.isna().sum()

# df_top_5_values_no_answers = num_nulls.sort_values(ascending=False)
df_top_5_values_with_no_answers = pd.DataFrame(num_nulls, columns=["Number of Non-Responses"]).reset_index()
df_top_5_values_with_no_answers.rename({"index" : "Question"}, axis=1, inplace=True)
df_top_5_values_with_no_answers["Percentage of Non-Responses"] = \
    (df_top_5_values_with_no_answers["Number of Non-Responses"] / len(df) * 100).round(2).astype(str) + "%"

df_top_5_values_with_no_answers.sort_values("Number of Non-Responses", ascending=False, inplace=True)
df_top_5_values_with_no_answers = df_top_5_values_with_no_answers.head(5)

In [36]:
# Write out num nulls to a file
obj = zip(num_nulls.index.tolist(), num_nulls.tolist())
nulls_list = list(obj)

with open("nulls.txt", mode="w", encoding="utf-8") as f:
    f.write("\n".join([f"{tup[0]}: {tup[1]}" for tup in nulls_list]))

In [9]:
fig = go.Figure(
    data=[go.Table(
        header=dict(values=["Question", "No Response", "Percentage of Non-Responses"],
                    align="left"),
        cells=dict(values=[df_top_5_values_with_no_answers.Question,
                       df_top_5_values_with_no_answers["Number of Non-Responses"],
                       df_top_5_values_with_no_answers["Percentage of Non-Responses"]],
                       align='left'))],
)
fig.update_layout(height=320, margin_t=20)
fig.show()

## What Program Were You in First Year?

In [10]:
# TO-DO! Make trace subplots with this
fig = px.pie(df, names="What program were you in first year?", color="What program were you in first year?")
fig.update_traces(textinfo="label+percent",
                  hovertemplate="<b>%{label}</b><br><i>Count</i>: %{value}")
fig.show()

## What Program Are You in Now?

In [11]:
fig = px.pie(df, names="What program are you in now?", color="What program are you in now?")
fig.update_traces(textinfo="label+percent",
                  hovertemplate="<b>%{label}</b><br><i>Count</i>: %{value}")
fig.show()

## What 2 Areas Did You Specialize In?

In [12]:
# df["What 2 areas did you specialize in?"].unique()

In [13]:
# Sort specialized areas to remove duplicates. (Ex. Area 6 + Area 5 is equivalent to Area 5 + Area 6)
df["What 2 areas did you specialize in?"] = df["What 2 areas did you specialize in?"].str.split(', ').map(lambda x: ", ".join(sorted(x)))

In [14]:
df_specialization = df.loc[:, ["What 2 areas did you specialize in?"]]
df_specialization.sort_values("What 2 areas did you specialize in?", ascending=False, inplace=True)

# hover text
areas_long_form = df_specialization["What 2 areas did you specialize in?"].str.split(", ").map(lambda x: " + ".join(x)).tolist()

# Simplified areas
areas_simplified = df_specialization["What 2 areas did you specialize in?"].map(
    lambda word: " + ".join(re.findall('(Area.*?):', word))
)

df_specialization["What 2 areas did you specialize in?"] = areas_simplified




In [15]:
# df.groupby("What 2 areas did you specialize in?", as_index=False).count()[["What 2 areas did you specialize in?", "Submission ID"]]
fig = px.pie(df_specialization, names="What 2 areas did you specialize in?", color="What 2 areas did you specialize in?")
fig.update_traces(textinfo="label+percent",
                  hovertemplate="%{text}<br><i>Count:</i> %{value}<br> %{percent}",
                  hovertext=areas_long_form,
                  text=areas_simplified)
fig.update_layout(legend_traceorder="reversed")
fig.show()

## What is Your Gender?

In [16]:
# gen_percent = ((df["What is your gender?"].value_counts() / df["What is your gender?"].count() * 100).round(2).astype(str) + "%").tolist()

In [22]:
fig = px.histogram(df, x="What is your gender?", color="What is your gender?")
fig.update_layout(xaxis_title="")
fig.update_traces(hovertemplate="%{x}=%{y}")
fig.show()

## Sexual Orientation Based on Gender

In [51]:
fig = px.pie(df, names="What is your sexual orientation?", facet_col="What is your gender?", 
                   facet_col_spacing=0.04,
                   color="What is your sexual orientation?")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_traces(hovertemplate="<i>%{label}</i><br>Count: %{value}")
fig.update_layout(showlegend=False)
fig.show()

## Are You a Domestic or International Student?

In [44]:
fig = px.histogram(df, x="Are you a domestic or international student?", color="Are you a domestic or international student?")
fig.update_layout(xaxis_title="")
fig.update_traces(hovertemplate="%{x}=%{y}")
fig.show()

## Where did you attend high school?

In [45]:
# df["Where did you attend high school?"].unique()

array(['Asia', 'GTA', 'West Coast of Canada', nan, 'Americas',
       'Southern Ontario', 'USA'], dtype=object)

In [67]:
df_high_school = pd.DataFrame(df["Where did you attend high school?"].value_counts()).reset_index()
# df_high_school.head(5)

,index,Where did you attend high school?
0,GTA,24
1,Asia,9
2,Southern Ontario,8
3,West Coast of Canada,7
4,Americas,1


In [4]:
df_traffic = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_traffic['text'] = df_traffic['airport'] + '' + df_traffic['city'] + ', ' + df_traffic['state'] + '' + 'Arrivals: ' + df_traffic['cnt'].astype(str)

fig = go.Figure(data=go.Scattergeo(
        lon = df_traffic['long'],
        lat = df_traffic['lat'],
        text = df_traffic['text'],
        mode = 'markers',
        marker_color = df_traffic['cnt'],
        ))


fig.update_geos(projection_type="orthographic")
fig.update_layout(height=600,
                  title = 'Most trafficked US airports<br>(Hover for airport names)',
                  )
# fig.update_layout(

# )
fig.show()

## Test

In [18]:
# a = pd.DataFrame([[True, False, False, False], [False, True, True, False], [True, True, False, False]], columns=[""])
# a.head()

In [ ]:
# Correlation Matrix could be fun to unravel some insight
# df.corr()